In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import functools
import glob
import logging
import numpy as np
import os
import torch
import importlib as imp

from tqdm import tqdm
tprint = tqdm.write

from torch_geometric.data import Data
from torch_geometric.nn.pool import radius_graph
from torch_geometric.transforms import FixedPoints
from tqdm import tqdm
from typing import Callable, Dict, List, Optional, Union

from typing import Callable, Optional, Union

import torch
from torch import Tensor
from torch.nn import Linear

# from torch_geometric.nn.conv import PointNetConv
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import reset
from torch_geometric.typing import (
    Adj,
    OptTensor,
    PairOptTensor,
    PairTensor
)
from torch_geometric.utils import add_self_loops, remove_self_loops
from torch_geometric.nn.norm import BatchNorm
import pytorch_lightning as pl
import pandas as pd

import aegnn
# from aegnn.models.networks.my_fuse import MyConvBNReLU


pl.seed_everything(12345)
device = torch.device('cuda')

torch.set_printoptions(precision=6)

# path = "/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230328183028/epoch=0-step=202.pt" #fuse
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331161858/epoch=18-step=3856.pt' # fuse, quant test
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331160055/epoch=12-step=2638.pt' # fuse, pos div 32
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331164223/epoch=11-step=2435.pt' # fuse, pos abs
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331230610/epoch=99-step=20299.pt' # quant test
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230331220429/epoch=1-step=405.pt' # quant debug: dpos = 0
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230406184251/epoch=17-step=3653.pt' # quant debug
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230406014814/epoch=99-step=20299.pt' # quant debug 2
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230407154758/epoch=99-step=20299.pt' # quant debug + qLinear
path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230409231150/epoch=99-step=20299.pt' # quant debug + qLinear 2
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230410051217/epoch=99-step=20299.pt' # quant debug + qLinear + sum_pool
# path = '/users/yyang22/thesis/aegnn_project/aegnn_results/training_results/checkpoints/ncars/recognition/20230410035349/epoch=99-step=20299.pt' # quant debug + qLinear + avg_pool
model = torch.load(path).to(device)
model.eval()
dm = aegnn.datasets.NCars(batch_size=1, shuffle=False)
dm.setup()
# print(model.model.fuse1.local_nn.weight)
# data_loader = dm.val_dataloader(num_workers=1).__iter__()
data_loader = dm.val_dataloader(num_workers=1)

assert model.model.fuse1.local_nn.bias is None

if isinstance(model, pl.LightningModule):
    nn_model = model._modules['model']
    nn_layers = nn_model._modules
elif isinstance(model, torch.nn.Module):
    nn_layers = model._modules
else:
    raise TypeError(f'The type of model is {type(model)}, not a `torch.nn.Module` or a `pl.LightningModule`')

from copy import deepcopy
unfused_model = deepcopy(model.model)
unfused_model = unfused_model.to(model.device)
unfused_model.eval()

# for key, nn in nn_layers.items():
#     if isinstance(nn, aegnn.models.networks.my_fuse.MyConvBNReLU):
#         # nn_layers[key].module.running_mean = torch.zeros_like(nn_layers[key].module.running_mean)
#         # nn_layers[key].module.running_var = torch.ones_like(nn_layers[key].module.running_var)
#         # nn_layers[key].module.bias = torch.nn.Parameter(torch.zeros_like(nn_layers[key].module.bias))
#         # nn_layers[key].module.weight = torch.nn.Parameter(torch.ones_like(nn_layers[key].module.weight))
#         # nn_layers[key].module.eps = 1e-16
#         nn_layers[key].to_fused()
#         pass
# fused_model = model
# fused_model.eval()
assert model.model.fused is False
assert model.model.quantized is False
model.model.to_fused()
assert model.model.fused is True
assert model.model.quantized is False

model.model.debug_logger()

fused_model = model
fused_model.eval()



# num_test_samples = 12
num_test_samples = 2460
with torch.no_grad():
    for i, sample in enumerate(tqdm(data_loader, position=1, desc='Samples', total=num_test_samples)):
        torch.cuda.empty_cache()
        if i==num_test_samples: break
        # tprint(f"\nSample {i}, file_id {sample.file_id}:")

        sample = sample.to(model.device)
        tot_nodes = sample.num_nodes


        unfused_test_sample = sample.clone().detach()
        output_unfused = unfused_model.forward(unfused_test_sample)
        y_unfused = torch.argmax(output_unfused, dim=-1)
        # tprint(f'unfused output = {output_unfused}')
        # tprint(f'{unfused_model.fuse1.local_nn.weight}')

        fused_test_sample = sample.clone().detach()
        output_fused = fused_model.forward(fused_test_sample)
        y_fused = torch.argmax(output_fused, dim=-1)
        # tprint(f'  fused output = {output_fused}')
        # tprint(f'{fused_model.model.fuse1.local_nn.weight}')
        # tprint(fused_model.model.fuse1.fused)

        diff = torch.allclose(y_unfused, y_fused)
        if diff is not True:
            print(i)
            print(f'unfused output = {output_unfused}')
            print(f'  fused output = {output_fused}')
        # print(diff)




/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 12345
Samples: 100%|██████████| 2460/2460 [00:30<00:00, 79.69it/s]


In [2]:
from aegnn.models.networks.my_fuse import MyConvBNReLU, qLinear

for block in fused_model.model.children():
    if isinstance(block,MyConvBNReLU):

        # print(f'x max = {block.obs_x.max_val}')
        # print(f'x min = {block.obs_x.min_val}')
        # x_scale, _ = block.obs_x.calculate_qparams()
        # print(f'x scale = \n{x_scale}')
        tot_x_max = torch.max(block.obs_x.max_val)
        print(f'tot x max = {tot_x_max}')

        # print(f'y max = {block.obs_y.max_val}')
        tot_y_max = torch.max(block.obs_y.max_val)
        print(f'tot y max = {tot_y_max}')

        # print(f'w max = {block.obs_w.max_val}')
        # print(f'w min = {block.obs_w.min_val}')
        # # # w_scale, _ = block.obs_w.calculate_qparams()
        # # # print(f'w scale = \n{w_scale}')
        # tot_w_max = torch.max(block.obs_w.max_val)
        # tot_w_min = torch.min(block.obs_w.min_val)
        # print(f'tot w max = {tot_w_max}')
        # print(f'tot w min = {tot_w_min}')
        tot_wx_max = torch.max(block.obs_w.max_val[:-2])
        tot_wpos_max = torch.max(block.obs_w.max_val[-2:])
        tot_wx_min = torch.min(block.obs_w.min_val[:-2])
        tot_wpos_min = torch.min(block.obs_w.min_val[-2:])
        # print(f'tot wx   max = {tot_wx_max}, min = {tot_wx_min}')
        # print(f'tot wpos max = {tot_wpos_max}, min = {tot_wpos_min}')
        tot_wx_abs_max = torch.maximum(torch.abs(tot_wx_max), torch.abs(tot_wx_min))
        tot_wpos_abs_max = torch.maximum(torch.abs(tot_wpos_max), torch.abs(tot_wpos_min))
        print(f'tot wx   abs max = {tot_wx_abs_max}')
        print(f'tot wx abs max * x max= {tot_wx_abs_max*tot_x_max}')
        print(f'tot wpos abs max = {tot_wpos_abs_max}')




        tot_b_max = torch.max(block.obs_b.max_val)
        tot_b_min = torch.min(block.obs_b.min_val)
        tot_b_abs_max = torch.maximum(tot_b_max.abs(), tot_b_min.abs())
        # print(f'tot b max = {tot_b_max}')
        # print(f'tot b min = {tot_b_min}')
        print(f'tot b abs max = {tot_b_abs_max}')
        
        print(f'calibre = {block.calibre}')
        # print(f'x_scale = {block.x_scale}')
        # print(f'y_scale = {block.y_scale}')
        # print(f'w_scale = {block.w_scale}')
        # print(f'b_scale = {block.b_scale}')
        # print(f'M = {block.M}')
        
        print('')

        # print(f'w min = {block.local_nn.weight}')

    if isinstance(block,qLinear):
        print(f'qLinear:')
        print(f'tot in max = {block.obs_in.max_val}')
        print(f'tot out max = {block.obs_out.max_val}')
        print(f'tot w max = {block.obs_w.max_val}')
        print(f'tot w min = {block.obs_w.min_val}')
        print(f'calibre = {block.calibre}')



tot x max = 1.0
tot y max = 2.5338151454925537
tot wx   abs max = 2.4780657291412354
tot wx abs max * x max= 2.4780657291412354
tot wpos abs max = 4.607081413269043
tot b abs max = 3.6142735481262207
calibre = True

tot x max = 2.5338151454925537
tot y max = 6.525791168212891
tot wx   abs max = 2.2445249557495117
tot wx abs max * x max= 5.687211513519287
tot wpos abs max = 1.9054051637649536
tot b abs max = 4.104064464569092
calibre = True

tot x max = 6.525791168212891
tot y max = 6.138178825378418
tot wx   abs max = 1.696285367012024
tot wx abs max * x max= 11.06960391998291
tot wpos abs max = 1.0249121189117432
tot b abs max = 2.489299774169922
calibre = True

tot x max = 6.138178825378418
tot y max = 9.349170684814453
tot wx   abs max = 1.3518762588500977
tot wx abs max * x max= 8.29805850982666
tot wpos abs max = 1.1923547983169556
tot b abs max = 3.853440999984741
calibre = True

qLinear:
tot in max = 10.49801254272461
tot out max = 46.62775421142578
tot w max = 0.654875338077545

In [3]:
assert fused_model.model.quantized is False

f_bit = 8
conv_w_bit = 8
fc_w_bit = 8
fc_out_bit = 8

fused_model.model.quant(conv_f_dtype=(f_bit,False), conv_w_dtype=(conv_w_bit,True), fc_in_dtype=(f_bit,False), fc_w_dtype=(fc_w_bit,True), fc_out_dtype=(fc_out_bit,True))

print(f'conv: f_dtype:{fused_model.model.fuse1.f_dtype}, w_dtype:{fused_model.model.fuse1.w_dtype}')
print(f'fc: in_dtype:{fused_model.model.fc.in_dtype}, out_dtype:{fused_model.model.fc.out_dtype}, w_dtype:{fused_model.model.fc.w_dtype}')

conv: f_dtype:uint8, w_dtype:int8
fc: in_dtype:uint8, out_dtype:int8, w_dtype:int8


In [ ]:
assert fused_model.model.quantized is True
assert fused_model.model.fuse1.quantized is True
assert fused_model.model.fc.quantized is True

In [ ]:
for block in fused_model.model.children():
    if isinstance(block,MyConvBNReLU):
        
        print(f'quantized = {block.quantized}')
        print(f'x_scale = {block.x_scale}')
        print(f'y_scale = {block.y_scale}')
        # print(f'w_scale = {block.w_scale}')
        print(f'dpos_scale = {block.dpos_scale}')
        print(f'1/dpos_scale = {1/block.dpos_scale}')
        # print(f'1/round(1/dpos_scale) = {(1/torch.round(1/block.dpos_scale))}')
        # print(f'wx_scale = {block.wx_scale}')
        # print(f'wpos_scale = {block.wpos_scale}')
        print(f'b_scale = {block.b_scale}')
        print(f'M = {block.M}')
        m = torch.round(block.M * 2**(20))
        print(f'm = {m}')
        print(f'M<<>> = {2**(-20)*m}')

        # print(f'b_new = {block.b_new}')
        # print(f'b_quant = {block.b_quant}')

        # print(f'w_quant = {block.w_quant}')
        
        print('')

        # print(f'w min = {block.local_nn.weight}')

In [ ]:
model.model.debug_y

In [4]:
quant_model = model
print('start quant test')
# model.model.debug_logger()
# num_test_samples = 2460
# num_test_samples = 12

unfused_correct = 0
quant_correct = 0
with torch.no_grad():
    # for i, sample in enumerate(tqdm(data_loader, position=1, desc='Samples', total=num_test_samples)):
    for i, sample in enumerate(data_loader):
        torch.cuda.empty_cache()
        if i==num_test_samples: break
        # tprint(f"\nSample {i}, file_id {sample.file_id}:")

        sample = sample.to(model.device)
        tot_nodes = sample.num_nodes


        unfused_test_sample = sample.clone().detach()
        output_unfused = unfused_model.forward(unfused_test_sample)
        y_unfused = torch.argmax(output_unfused, dim=-1)
        # print(f'unfused output = {output_unfused}')
        # print(f'x = {unfused_test_sample.x}')
        # tprint(f'{unfused_model.fuse1.local_nn.weight}')
        unfused_hit = torch.allclose(y_unfused, unfused_test_sample.y)
        if unfused_hit: unfused_correct += 1

        quant_test_sample = sample.clone().detach()
        output_quant = quant_model.forward(quant_test_sample)
        y_quant = torch.argmax(output_quant, dim=-1)
        # print(f'  quant output = {output_quant}')
        # print(f'x = {quant_test_sample.x}')
        # tprint(f'{quant_model.model.fuse1.local_nn.weight}')
        # tprint(quant_model.model.fuse1.fused)

        quant_hit = torch.allclose(y_quant, quant_test_sample.y)
        if quant_hit: quant_correct += 1

        # diff = torch.allclose(y_unfused, y_quant)
        # if diff is not True:
        #     print(i)
        #     print(f'unfused output = {output_unfused}')
        #     print(f'  quant output = {output_quant}')
            # print(unfused_test_sample.x)
            # print(quant_test_sample.x)
        if i == 3 :
            print('debug:')
            print(i)
            print(f'unfused output = {output_unfused}')
            print(f'  quant output = {output_quant}')

unfused_acc = unfused_correct / num_test_samples
quant_acc = quant_correct / num_test_samples

print(f'unfused_acc = {unfused_acc}')
print(f'quant_acc = {quant_acc}')

print(f'conv: f_dtype:{fused_model.model.fuse1.f_dtype}, w_dtype:{fused_model.model.fuse1.w_dtype}')
print(f'fc: in_dtype:{fused_model.model.fc.in_dtype}, w_dtype:{fused_model.model.fc.w_dtype}, out_dtype:{fused_model.model.fc.out_dtype}')

start quant test
debug:
3
unfused output = tensor([[ 14.957020, -15.114644]], device='cuda:0')
  quant output = tensor([[ 74271., -75538.]], device='cuda:0')
unfused_acc = 0.9605691056910569
quant_acc = 0.9613821138211383
conv: f_dtype:uint8, w_dtype:int8
fc: in_dtype:uint8, w_dtype:int8, out_dtype:int8


unfused acc = 0.9605691056910569

config: conv f_dtype, w_dtype; fc in_dtype, w_dtype, out_dtype

uint8, int8; uint8, int8, int8: 0.9617886178861789

uint7, int8; uint7, int8, int8: 0.9601626016260163

------

uint8, int7; uint8, int8, int8: 0.9617886178861789

uint8, int6; uint8, int8, int8: 0.9451219512195121

uint8, int5; uint8, int8, int8: 0.8902439024390244

uint8, int4; uint8, int8, int8: 0.8951219512195122

uint8, int3; uint8, int8, int8: 0.7223577235772358

uint8, int2; uint8, int8, int8: 0.4739837398373984

-------

uint8, int8; uint8, int7, int8: 0.9601626016260163

uint8, int8; uint8, int6, int8: 0.9609756097560975

uint8, int8; uint8, int5, int8: 0.9613821138211383

uint8, int8; uint8, int4, int8: 0.958130081300813

uint8, int8; uint8, int3, int8: 0.9508130081300813

uint8, int8; uint8, int2, int8: 0.598780487804878


In [ ]:
model.model.debug_dqy

In [ ]:
# model.model.debug_qy['fuse4'].max()
# model.model.debug_dqy['fuse4'].max()
# model.model.debug_y['fuse4'].max()
# model.model.debug_y['fuse4'].max() / model.model.fuse4.y_scale
model.model.fuse4.y_scale

In [ ]:
fc_qin = model.model.debug_fc['qin']
fc_in = model.model.debug_fc['in']
print(fc_in.max())
print(fc_qin.max())
print(fc_in.max()/fc_qin.max())



fc_qout = model.model.debug_qy['qout']
fc_out = model.model.debug_fc['out']
print(fc_out.max())
print(fc_qout.max())
print(fc_out.min())
print(fc_qout.min())

In [ ]:
for nn in quant_model.model.children():
    if isinstance(nn, MyConvBNReLU):
        w_quant = nn.local_nn.weight
        b_quant = nn.b_quant
        print(w_quant)
        print(b_quant)